In [1]:
import pandas as pd
import numpy as np
import re
import json
from collections import defaultdict
import math

In [2]:
pickle_path = "/Users/saurabhkumar/Desktop/Work/Tennis"
atp_data     = pd.read_pickle(f"{pickle_path}/ATPdata1968_2024.pkl")

In [3]:
atp_data.columns

Index(['tourney_id', 'tourney_name', 'surface', 'draw_size', 'tourney_level',
       'tourney_date', 'match_num', 'winner_id', 'winner_seed', 'winner_entry',
       'winner_name', 'winner_hand', 'winner_ht', 'winner_ioc', 'winner_age',
       'loser_id', 'loser_seed', 'loser_entry', 'loser_name', 'loser_hand',
       'loser_ht', 'loser_ioc', 'loser_age', 'score', 'best_of', 'round',
       'minutes', 'w_ace', 'w_df', 'w_svpt', 'w_1stIn', 'w_1stWon', 'w_2ndWon',
       'w_SvGms', 'w_bpSaved', 'w_bpFaced', 'l_ace', 'l_df', 'l_svpt',
       'l_1stIn', 'l_1stWon', 'l_2ndWon', 'l_SvGms', 'l_bpSaved', 'l_bpFaced',
       'winner_rank', 'winner_rank_points', 'loser_rank', 'loser_rank_points'],
      dtype='object')

In [ ]:
atp_data.tail(10)

In [4]:
# Extract year and combine winner/loser data into single player dataset
atp_data['year'] = atp_data['tourney_date'].astype(str).str[:4].astype(int)

players = pd.concat([
    atp_data[['winner_name', 'winner_ioc', 'winner_rank', 'year']].rename(columns={'winner_name': 'name', 'winner_ioc': 'country', 'winner_rank': 'rank'}),
    atp_data[['loser_name', 'loser_ioc', 'loser_rank', 'year']].rename(columns={'loser_name': 'name', 'loser_ioc': 'country', 'loser_rank': 'rank'})
]).dropna(subset=['name', 'country', 'rank'])

print(f"Total player appearances: {len(players):,}")

Total player appearances: 306,751


In [5]:
# Get each player's best (lowest) rank per year - fixes duplication issue
best_ranks = players.groupby(['year', 'name', 'country'])['rank'].min().reset_index()

best_ranks['name'] = best_ranks['name'].str.strip().str.title()
best_ranks['country'] = best_ranks['country'].str.strip().str.upper()

print(f"Data from {best_ranks['year'].min()} to {best_ranks['year'].max()}")
print(f"Unique player-year records: {len(best_ranks):,}")
best_ranks.head()

Data from 1973 to 2024
Unique player-year records: 19,262


,year,name,country,rank
0,1973,Adriano Panatta,ITA,8.0
1,1973,Allan Stone,AUS,55.0
2,1973,Anand Amritraj,IND,99.0
3,1973,Andrew Pattison,USA,52.0
4,1973,Antonio Munoz,ESP,74.0


In [6]:
# Build country rankings data structure
result = {}
for year in sorted(best_ranks['year'].unique()):
    year_data = best_ranks[best_ranks['year'] == year]
    country_stats = {}
    
    for country in year_data['country'].unique():
        country_players = year_data[year_data['country'] == country]
        
        # Count unique players ever in top 100 and top 10 during the year
        ever_in_top100 = int((country_players['rank'] <= 100).sum())
        ever_in_top10 = int((country_players['rank'] <= 10).sum())
        
        # Find top player (best rank)
        top_idx = country_players['rank'].idxmin()
        top_player = country_players.loc[top_idx, 'name']
        top_rank = int(country_players.loc[top_idx, 'rank'])
        
        country_stats[country] = {
            'ever_in_top100': ever_in_top100,
            'ever_in_top10': ever_in_top10,
            'top_player': top_player,
            'top_rank': top_rank
        }
    result[year] = country_stats

print(f"Processed {len(result)} years")

Processed 52 years


In [7]:
# Strategic year sampling for better visualization
selected_years = [1970, 1975, 1980, 1985, 1990, 1995, 2000, 2003, 2006, 2009, 2012, 2015, 2018, 2021, 2024]
filtered_result = {str(year): result[year] for year in selected_years if year in result}

print(f"Filtered to {len(filtered_result)} years: {list(filtered_result.keys())}")

Filtered to 14 years: ['1975', '1980', '1985', '1990', '1995', '2000', '2003', '2006', '2009', '2012', '2015', '2018', '2021', '2024']


In [9]:
print(filtered_result)

{'1975': {'ITA': {'ever_in_top100': 4, 'ever_in_top10': 0, 'top_player': 'Adriano Panatta', 'top_rank': 28}, 'RUS': {'ever_in_top100': 1, 'ever_in_top10': 0, 'top_player': 'Alex Metreveli', 'top_rank': 13}, 'AUS': {'ever_in_top100': 19, 'ever_in_top10': 3, 'top_player': 'John Newcombe', 'top_rank': 2}, 'COL': {'ever_in_top100': 1, 'ever_in_top10': 0, 'top_player': 'Ivan Molina', 'top_rank': 77}, 'EGY': {'ever_in_top100': 1, 'ever_in_top10': 0, 'top_player': 'Ismail El Shafei', 'top_rank': 34}, 'IND': {'ever_in_top100': 3, 'ever_in_top10': 0, 'top_player': 'Vijay Amritraj', 'top_rank': 31}, 'USA': {'ever_in_top100': 31, 'ever_in_top10': 3, 'top_player': 'Jimmy Connors', 'top_rank': 1}, 'ESP': {'ever_in_top100': 4, 'ever_in_top10': 1, 'top_player': 'Manuel Orantes', 'top_rank': 7}, 'GER': {'ever_in_top100': 6, 'ever_in_top10': 0, 'top_player': 'Karl Meiler', 'top_rank': 38}, 'HUN': {'ever_in_top100': 1, 'ever_in_top10': 0, 'top_player': 'Balazs Taroczy', 'top_rank': 38}, 'CHI': {'ever_in

In [ ]:
# ============================================================================
# HELPER FUNCTION: Calculate Herfindahl Index (separate for easy refinement)
# ============================================================================
def calculate_shannon_index(country_counts):
    """
    Calculate Shannon Diversity Index for distribution diversity
    Shannon = -sum(p_i * log(p_i)) where p_i is proportion of each country
    Range: 0 (no diversity) to log(n) (maximum diversity)
    Higher values = more diverse/global
    """
    total = sum(country_counts.values())
    if total == 0:
        return 0

    shannon = 0
    for count in country_counts.values():
        if count > 0:  # Only include countries with players
            proportion = count / total
            shannon += proportion * math.log(proportion)

    return round(-shannon, 4)  # Negative because we want positive values

# ============================================================================
# DATASET 1: Global Tennis Timeline (Powers 3 charts)
# ============================================================================
def create_global_timeline_dataset(filtered_result, top_n_countries=8):
    """
    Create comprehensive timeline dataset for multiple visualizations
    """
    timeline_data = []
    
    for year_str in sorted(filtered_result.keys(), key=int):
        year_int = int(year_str)
        year_data = filtered_result[year_str]
        
        # Get top 100 and top 10 counts per country
        top100_counts = {country: stats['ever_in_top100'] 
                        for country, stats in year_data.items()}
        top10_counts = {country: stats['ever_in_top10'] 
                       for country, stats in year_data.items() if stats['ever_in_top10'] > 0}
        
        # Sort countries by top 100 count for consistent ordering
        sorted_countries_top100 = sorted(top100_counts.items(), key=lambda x: x[1], reverse=True)
        
        # Create top N countries list, group rest as "Other"
        countries_top100 = []
        other_count_top100 = 0
        
        for i, (country, count) in enumerate(sorted_countries_top100):
            if i < top_n_countries:
                countries_top100.append({"country": country, "count": count})
            else:
                other_count_top100 += count
        
        if other_count_top100 > 0:
            countries_top100.append({"country": "Other", "count": other_count_top100})
        
        # Create top 10 countries list
        countries_top10 = []
        other_count_top10 = 0
        sorted_countries_top10 = sorted(top10_counts.items(), key=lambda x: x[1], reverse=True)
        
        for i, (country, count) in enumerate(sorted_countries_top10):
            if i < top_n_countries:
                countries_top10.append({"country": country, "count": count})
            else:
                other_count_top10 += count
        
        if other_count_top10 > 0:
            countries_top10.append({"country": "Other", "count": other_count_top10})
        
        # Calculate totals and metrics
        total_top100 = sum(top100_counts.values())
        total_top10 = sum(top10_counts.values())
        num_countries_top100 = len([c for c in top100_counts.values() if c > 0])
        num_countries_top10 = len([c for c in top10_counts.values() if c > 0])
        
        # Calculate shannon
        shannon_top100 = calculate_shannon_index(top100_counts)
        shannon_top10 = calculate_shannon_index(top10_counts)
        
        year_entry = {
            "year": year_int,
            "countries_top100": countries_top100,
            "countries_top10": countries_top10,
            "num_countries_top100": num_countries_top100,
            "num_countries_top10": num_countries_top10,
            "total_players_top100": total_top100,
            "total_players_top10": total_top10,
            "concentration_index_top100": hhi_top100,
            "concentration_index_top10": hhi_top10
        }
        
        timeline_data.append(year_entry)
    
    return timeline_data

# ============================================================================
# DATASET 2: Country Tennis Milestones (Powers 1 chart)
# ============================================================================
def create_country_milestones_dataset(filtered_result):
    """
    Create dataset showing when each country first appeared in top 100/10
    """
    country_milestones = {}
    
    # Process years in chronological order
    for year_str in sorted(filtered_result.keys(), key=int):
        year_int = int(year_str)
        year_data = filtered_result[year_str]
        
        for country, stats in year_data.items():
            if country not in country_milestones:
                country_milestones[country] = {
                    "country": country,
                    "first_top100_year": None,
                    "first_top10_year": None
                }
            
            # Record first appearance in top 100
            if (stats['ever_in_top100'] > 0 and 
                country_milestones[country]["first_top100_year"] is None):
                country_milestones[country]["first_top100_year"] = year_int
            
            # Record first appearance in top 10
            if (stats['ever_in_top10'] > 0 and 
                country_milestones[country]["first_top10_year"] is None):
                country_milestones[country]["first_top10_year"] = year_int
    
    # Convert to list and sort by first top 100 appearance
    milestones_list = list(country_milestones.values())
    milestones_list.sort(key=lambda x: x["first_top100_year"] or 9999)
    
    return milestones_list

# ============================================================================
# DATASET 3: Country Tennis Profiles (Powers 1 chart)
# ============================================================================
def create_country_profiles_dataset(filtered_result):
    """
    Create comprehensive country profiles for depth vs performance analysis
    """
    country_profiles = {}
    
    for year_str, year_data in filtered_result.items():
        year_int = int(year_str)
        
        for country, stats in year_data.items():
            if country not in country_profiles:
                country_profiles[country] = {
                    "country": country,
                    "total_top100_appearances": 0,
                    "total_top10_appearances": 0,
                    "best_rank_ever": 999,
                    "years_with_top100": 0,
                    "years_with_top10": 0,
                    "years_active": 0
                }
            
            profile = country_profiles[country]
            profile["years_active"] += 1
            profile["total_top100_appearances"] += stats['ever_in_top100']
            profile["total_top10_appearances"] += stats['ever_in_top10']
            
            if stats['ever_in_top100'] > 0:
                profile["years_with_top100"] += 1
            
            if stats['ever_in_top10'] > 0:
                profile["years_with_top10"] += 1
            
            if stats['top_rank'] < profile["best_rank_ever"]:
                profile["best_rank_ever"] = stats['top_rank']
    
    # Calculate averages and convert to list
    profiles_list = []
    for country, profile in country_profiles.items():
        profile["avg_top100_per_year"] = round(
            profile["total_top100_appearances"] / profile["years_active"], 1
        ) if profile["years_active"] > 0 else 0
        
        profile["avg_top10_per_year"] = round(
            profile["total_top10_appearances"] / profile["years_active"], 1
        ) if profile["years_active"] > 0 else 0
        
        profiles_list.append(profile)
    
    # Sort by total top 100 appearances
    profiles_list.sort(key=lambda x: x["total_top100_appearances"], reverse=True)
    
    return profiles_list

print("Dataset generation functions created successfully!")
print("Ready to generate the three datasets from your filtered_result variable.")

In [ ]:
# ============================================================================
# GENERATE ALL THREE DATASETS
# ============================================================================

print("=" * 60)
print("GENERATING DATASET 1: Global Tennis Timeline")
print("=" * 60)

# Generate Dataset 1: Global Tennis Timeline
timeline_data = create_global_timeline_dataset(filtered_result, top_n_countries=8)

# Save Dataset 1
with open('global_tennis_timeline.json', 'w') as f:
    json.dump(timeline_data, f, indent=2)

print(f"✅ Dataset 1 saved: global_tennis_timeline.json")
print(f"   - {len(timeline_data)} years of data")
print(f"   - Powers 3 charts: Tennis Goes Global, Rise and Fall of Empires, Democratization")
print(f"   - Sample entry for {timeline_data[0]['year']}:")
print(f"     • Countries in top 100: {timeline_data[0]['num_countries_top100']}")
print(f"     • Countries in top 10: {timeline_data[0]['num_countries_top10']}")
print(f"     • Concentration index (top 100): {timeline_data[0]['concentration_index_top100']}")

print("\n" + "=" * 60)
print("GENERATING DATASET 2: Country Tennis Milestones")
print("=" * 60)

# Generate Dataset 2: Country Tennis Milestones
milestones_data = create_country_milestones_dataset(filtered_result)

# Save Dataset 2
with open('country_tennis_milestones.json', 'w') as f:
    json.dump(milestones_data, f, indent=2)

print(f"✅ Dataset 2 saved: country_tennis_milestones.json")
print(f"   - {len(milestones_data)} countries tracked")
print(f"   - Powers 1 chart: New Tennis Nations Timeline")
print(f"   - Sample entries:")
for i in range(min(3, len(milestones_data))):
    country = milestones_data[i]
    print(f"     • {country['country']}: Top 100 since {country['first_top100_year']}, Top 10 since {country['first_top10_year']}")

print("\n" + "=" * 60)
print("GENERATING DATASET 3: Country Tennis Profiles")
print("=" * 60)

# Generate Dataset 3: Country Tennis Profiles
profiles_data = create_country_profiles_dataset(filtered_result)

# Save Dataset 3
with open('country_tennis_profiles.json', 'w') as f:
    json.dump(profiles_data, f, indent=2)

print(f"✅ Dataset 3 saved: country_tennis_profiles.json")
print(f"   - {len(profiles_data)} countries profiled")
print(f"   - Powers 1 chart: Tennis Depth vs Peak Performance")
print(f"   - Top 3 countries by total top 100 appearances:")
for i in range(min(3, len(profiles_data))):
    country = profiles_data[i]
    print(f"     • {country['country']}: {country['total_top100_appearances']} appearances, best rank #{country['best_rank_ever']}")

print("\n" + "=" * 60)
print("🎾 ALL DATASETS GENERATED SUCCESSFULLY! 🎾")
print("=" * 60)
print("Ready for JavaScript visualization on GitHub Pages!")
print("\nFiles created:")
print("1. global_tennis_timeline.json - Powers 3 charts")
print("2. country_tennis_milestones.json - Powers 1 chart") 
print("3. country_tennis_profiles.json - Powers 1 chart")
print("\nTotal: 3 datasets powering 5 compelling tennis visualizations! 🚀")

In [32]:
# Convert to array of years structure
years_array = []
for year in sorted(filtered_result.keys(), key=int):
    years_array.append({
        "year": int(year),
        "countries": filtered_result[year]
    })

# Save year-centric JSON
with open("/Users/saurabhkumar/Desktop/Work/Tennis/top100bycountry/top_tennis_players_timeline.json", 'w') as f:
    json.dump(years_array, f, indent=2)

print(f"Saved year-centric JSON with {len(years_array)} years")

Saved year-centric JSON with 14 years


In [33]:
# Build country-centric structure
country_centric = {}
for year in filtered_result:
    for country in filtered_result[year]:
        if country not in country_centric:
            country_centric[country] = {}
        country_centric[country][year] = filtered_result[year][country]

# Save country-centric JSON
with open("/Users/saurabhkumar/Desktop/Work/Tennis/top100bycountry/tennis_country_profiles.json", 'w') as f:
    json.dump(country_centric, f, indent=2)

print(f"Saved country-centric JSON with {len(country_centric)} countries")

Saved country-centric JSON with 107 countries


In [31]:
# Preview year-centric structure
print("Year-centric sample:")
print(json.dumps(years_array[0], indent=2)[:500] + "...")

print("\nCountry-centric sample:")
sample_country = list(country_centric.keys())[0]
print(f"Sample for {sample_country}:")
print(json.dumps({sample_country: country_centric[sample_country]}, indent=2)[:500] + "...")

Year-centric sample:
{
  "year": 1975,
  "countries": {
    "ITA": {
      "ever_in_top100": 4,
      "ever_in_top10": 0,
      "top_player": "Adriano Panatta",
      "top_rank": 28
    },
    "RUS": {
      "ever_in_top100": 1,
      "ever_in_top10": 0,
      "top_player": "Alex Metreveli",
      "top_rank": 13
    },
    "AUS": {
      "ever_in_top100": 19,
      "ever_in_top10": 3,
      "top_player": "John Newcombe",
      "top_rank": 2
    },
    "COL": {
      "ever_in_top100": 1,
      "ever_in_top10": 0,
   ...

Country-centric sample:
Sample for ITA:
{
  "ITA": {
    "1975": {
      "ever_in_top100": 4,
      "ever_in_top10": 0,
      "top_player": "Adriano Panatta",
      "top_rank": 28
    },
    "1980": {
      "ever_in_top100": 4,
      "ever_in_top10": 0,
      "top_player": "Corrado Barazzutti",
      "top_rank": 19
    },
    "1985": {
      "ever_in_top100": 3,
      "ever_in_top10": 0,
      "top_player": "Francesco Cancellotti",
      "top_rank": 21
    },
    "1990"